# Infini-gram Document Search: N-gram to Document Retrieval

This notebook demonstrates how to extract n-grams from input text and retrieve the top 3 documents containing each n-gram using the Infini-gram API.

## Workflow:
1. Extract n-grams from input text (2-gram to 4-gram)
2. For each n-gram, query the Infini-gram API to find matching documents
3. Retrieve the top 3 documents for each n-gram
4. Display results with document text and metadata

## Requirements:
- Access to Infini-gram API (https://api.infini-gram.io/)
- Python packages: requests, transformers (for tokenization)

## 1. Setup and Configuration

Import required packages and configure API settings.

In [57]:
import requests
import json
from typing import List, Dict, Any
from collections import defaultdict

# API configuration
API_BASE_URL = "https://api.infini-gram.io/"
SELECTED_INDEX = "v4_rpj_llama_s4"  # RedPajama with Llama-2 tokenizer

print("✅ Setup complete")
print(f"📡 API Base URL: {API_BASE_URL}")
print(f"📊 Using index: {SELECTED_INDEX}")

✅ Setup complete
📡 API Base URL: https://api.infini-gram.io/
📊 Using index: v4_rpj_llama_s4


## 2. API Access Verification

Test API connectivity and verify the selected index is working.

In [58]:
def check_api_access():
    """Verify API access with a simple test query"""
    try:
        payload = {
            'index': SELECTED_INDEX,
            'query_type': 'count',
            'query': 'the'
        }
        
        response = requests.post(API_BASE_URL, json=payload, timeout=10)
        
        if response.status_code == 200:
            result = response.json()
            if 'error' not in result:
                print(f"✅ API is working! Test count: {result.get('count', 'N/A'):,}")
                return True
        
        print(f"❌ API check failed: {response.status_code}")
        return False
        
    except Exception as e:
        print(f"❌ API error: {e}")
        return False

# Test API access
api_working = check_api_access()
if api_working:
    print("🚀 Ready to search documents!")
else:
    print("⚠️ API issues detected - some functions may not work")

✅ API is working! Test count: 332,456,956
🚀 Ready to search documents!


## 3. N-gram Extraction

Extract n-grams from input text for document search.

In [59]:

def simple_tokenize(text: str) -> List[str]:
    """Simple tokenization for demonstration purposes"""
    # Clean and tokenize the text
    return text.lower().split()


In [60]:

def extract_ngrams(text: str, min_n: int = 1, max_n: int = 5) -> Dict[int, List[str]]:
    """
    Extract n-grams from text using simple word-based tokenization
    
    Args:
        text: Input text to extract n-grams from
        min_n: Minimum n-gram size
        max_n: Maximum n-gram size
    
    Returns:
        Dictionary mapping n-gram size to list of n-grams
    """
    # Tokenize the text
    tokens = simple_tokenize(text)
    
    ngrams = {}
    
    for n in range(min_n, max_n + 1):
        ngram_list = []
        for i in range(len(tokens) - n + 1):
            ngram = ' '.join(tokens[i:i + n])
            ngram_list.append(ngram)
        ngrams[n] = ngram_list
    
    return ngrams


## 4. Document Retrieval

Core functionality to retrieve top 3 documents for each n-gram from input text.

In [64]:
def get_top_documents_for_ngrams(text: str, max_ngram_size: int = 4, top_k: int = 3) -> List[Dict]:
    """
    Extract n-grams from text and get top K documents for each n-gram
    
    Args:
        text: Input text to extract n-grams from
        max_ngram_size: Maximum n-gram size (default: 4)
        top_k: Number of top documents to retrieve per n-gram (default: 3)
    
    Returns:
        List of results, one per n-gram with top documents
    """
    # Extract n-grams
    ngrams = extract_ngrams(text, min_n=2, max_n=max_ngram_size)
    
    # Flatten n-grams and prioritize longer ones
    all_ngrams = []
    for n in sorted(ngrams.keys(), reverse=True):
        for ngram in ngrams[n][:5]:  # Limit to 5 per size
            if ngram not in all_ngrams:
                all_ngrams.append(ngram)
    
    print(f"🔍 Processing {len(all_ngrams)} unique n-grams")
    
    results = []
    
    for i, ngram in enumerate(all_ngrams, 1):
        print(f"\n📡 {i}/{len(all_ngrams)}: Searching '{ngram}'")
        
        try:
            # Step 1: Use 'find' query to get documents containing this n-gram
            find_payload = {
                'index': SELECTED_INDEX,
                'query_type': 'find',
                'query': ngram
            }
            
            response = requests.post(API_BASE_URL, json=find_payload, timeout=15)
            
            if response.status_code != 200:
                print(f"   ❌ Find query failed: {response.status_code}")
                continue
            
            result = response.json()
            if 'error' in result:
                print(f"   ❌ API Error: {result['error']}")
                continue
            
            count = result.get('cnt', 0)
            segments = result.get('segment_by_shard', [])
            
            print(f"   📊 Found {count:,} occurrences across {len(segments)} shards")
            
            if count == 0:
                print(f"   📭 No documents found")
                results.append({
                    'ngram': ngram,
                    'ngram_length': len(ngram.split()),
                    'total_count': count,
                    'documents_retrieved': 0,
                    'documents': []
                })
                continue
            
            # Step 2: Retrieve actual documents from the segments
            documents = []
            retrieved = 0
            
            for shard_idx, (start_rank, end_rank) in enumerate(segments):
                if retrieved >= top_k:
                    break
                
                # Try several ranks from this shard
                ranks_to_try = list(range(start_rank, min(start_rank + 5, end_rank + 1)))
                
                for rank in ranks_to_try:
                    if retrieved >= top_k:
                        break
                    
                    try:
                        # Try to get document by rank using the method we discovered works
                        doc_payload = {
                            'index': SELECTED_INDEX,
                            'query_type': 'get_doc_by_rank',
                            'query': ngram,  # Include the query as required
                            's': shard_idx,
                            'rank': rank
                        }
                        
                        doc_response = requests.post(API_BASE_URL, json=doc_payload, timeout=10)
                        
                        if doc_response.status_code == 200:
                            doc_result = doc_response.json()
                            
                            if 'error' not in doc_result:
                                # Extract document text from spans
                                doc_text = ""
                                if 'spans' in doc_result:
                                    spans = doc_result['spans']
                                    doc_text = " ".join([span[0] for span in spans if span and len(span) > 0 and span[0]])
                                
                                if doc_text.strip():  # Only add if we got actual text
                                    documents.append({
                                        'rank': rank,
                                        'shard': shard_idx,
                                        'text': doc_text,
                                        'text_preview': doc_text[:300] + "..." if len(doc_text) > 300 else doc_text,
                                        'doc_length': len(doc_text.split()),
                                        'doc_ix': doc_result.get('doc_ix', 'unknown')
                                    })
                                    retrieved += 1
                                    print(f"   ✅ Retrieved document {retrieved}/{top_k} (rank: {rank})")
                                    
                                    # Short break between requests to avoid overloading
                                    import time
                                    time.sleep(0.1)
                            else:
                                print(f"   ⚠️ Error in document result: {doc_result.get('error', 'unknown')}")
                        else:
                            print(f"   ⚠️ Document request failed: {doc_response.status_code}")
                            
                    except Exception as e:
                        print(f"   ⚠️ Error getting document at rank {rank}: {e}")
                        continue
            
            # Add result for this n-gram
            ngram_result = {
                'ngram': ngram,
                'ngram_length': len(ngram.split()),
                'total_count': count,
                'documents_retrieved': len(documents),
                'documents': documents
            }
            
            results.append(ngram_result)
            
            if len(documents) > 0:
                print(f"   ✅ Successfully retrieved {len(documents)} documents with text")
            else:
                print(f"   ⚠️ Found {count:,} occurrences but couldn't retrieve document text")
            
        except Exception as e:
            print(f"   ❌ Failed to process '{ngram}': {e}")
    
    print(f"\n📋 Search completed: {len(results)} n-grams processed")
    return results

### Note on Document Retrieval

The current implementation verifies that n-grams exist in the corpus and returns the counts. 
To retrieve actual document text, you would need to:

1. Use the `find` query to get document segments
2. Use appropriate API endpoints to retrieve document content by rank/shard
3. Extract text from the returned spans

The framework above shows the structure - you can extend the document retrieval part based on your specific API access and requirements.

### Understanding "Rank" in Infini-gram

In the Infini-gram API, **"rank"** refers to the position/index of a document within the corpus. Here's how it works:

1. **Global Document Ordering**: Every document in the corpus has a unique rank (position number)
2. **Shard-based Organization**: Documents are distributed across multiple shards for efficiency
3. **Segment Ranges**: When you search for an n-gram, the API returns `segment_by_shard` which contains:
   - `[start_rank, end_rank]` pairs for each shard
   - These define ranges of document ranks that contain your n-gram

**Example:**
```python
segments = [[232539299067, 232539338892], [232516639042, 232516678886]]
#            ^start_rank    ^end_rank      ^start_rank    ^end_rank
#            Documents 232539299067 to    Documents 232516639042 to  
#            232539338892 contain the      232516678886 contain the
#            n-gram in shard 0             n-gram in shard 1
```

**To retrieve a specific document**: You use `get_doc_by_rank` with:
- `s`: shard index (0, 1, 2, etc.)
- `rank`: specific document rank within that range
- `query`: the original n-gram (required by API)

In [62]:
def display_results(results: List[Dict]):
    """Display search results in a clean, readable format"""
    
    if not results:
        print("📭 No results found")
        return
    
    print(f"\n📊 DOCUMENT SEARCH RESULTS")
    print("=" * 80)
    
    for i, result in enumerate(results, 1):
        print(f"\n🔎 N-gram {i}: '{result['ngram']}'")
        print(f"   📈 Total occurrences: {result['total_count']:,}")
        print(f"   📄 Documents retrieved: {result['documents_retrieved']}")
        
        if result['documents']:
            print(f"   📝 Top documents:")
            
            for j, doc in enumerate(result['documents'], 1):
                print(f"\n      📄 Document {j}:")
                print(f"         🎯 Rank: {doc['rank']}, Shard: {doc['shard']}")
                print(f"         📏 Length: {doc['doc_length']} words")
                print(f"         📃 Text: {doc['text_preview']}")
        else:
            print(f"   ❌ No documents retrieved")
        
        print("-" * 60)

## 5. Example Usage

Test the document retrieval functionality with sample text.

In [65]:
# Example: Search for documents containing n-grams from this text
sample_text = """
Artificial intelligence and machine learning are transforming how we process natural language.
Neural networks can now understand context and generate human-like responses.
"""

print("🚀 Starting document retrieval...")
print(f"📝 Input text: {sample_text.strip()}")

# Get top 3 documents for each n-gram
results = get_top_documents_for_ngrams(sample_text, max_ngram_size=3, top_k=3)

# Display results
display_results(results)

🚀 Starting document retrieval...
📝 Input text: Artificial intelligence and machine learning are transforming how we process natural language.
Neural networks can now understand context and generate human-like responses.
🔍 Processing 10 unique n-grams

📡 1/10: Searching 'artificial intelligence and'
   📊 Found 3,640 occurrences across 4 shards
   📊 Found 3,640 occurrences across 4 shards
   ✅ Retrieved document 1/3 (rank: 232539306669)
   ✅ Retrieved document 1/3 (rank: 232539306669)
   ✅ Retrieved document 2/3 (rank: 232539306670)
   ✅ Retrieved document 2/3 (rank: 232539306670)
   ✅ Retrieved document 3/3 (rank: 232539306671)
   ✅ Successfully retrieved 3 documents with text

📡 2/10: Searching 'intelligence and machine'
   ✅ Retrieved document 3/3 (rank: 232539306671)
   ✅ Successfully retrieved 3 documents with text

📡 2/10: Searching 'intelligence and machine'
   📊 Found 360 occurrences across 4 shards
   📊 Found 360 occurrences across 4 shards
   ✅ Retrieved document 1/3 (rank: 196

## Summary

This notebook provides a clean, focused implementation for:

1. **N-gram Extraction**: Breaking input text into 2-4 gram phrases
2. **Document Search**: Using Infini-gram API to find documents containing each n-gram  
3. **Document Retrieval**: Getting the top 3 documents with actual text content for each n-gram
4. **Results Display**: Showing results in a clear, organized format

### Key Features:
- ✅ Extracts meaningful n-grams from any input text
- ✅ Queries the Infini-gram API with proper error handling
- ✅ Retrieves actual document text content (not just counts)
- ✅ Returns top 3 documents per n-gram with metadata
- ✅ Displays results in an easy-to-read format

### Usage:
```python
# Basic usage
results = get_top_documents_for_ngrams("Your text here", max_ngram_size=3, top_k=3)
display_results(results)
```

The notebook is now ready for production use for n-gram-based document discovery and retrieval.